In [1]:
%reload_ext autoreload
%autoreload 2

%matplotlib inline

import sys
sys.path.append('../src')

import pandas as pd

# SB dataset

In [15]:
from concurrent.futures import ProcessPoolExecutor
from pathlib import Path

from imfusion.insertions.aligners.star import read_chimeric_junctions
from nbsupport.util import flagstat


def read_stats(base_path, transposon_name, n_cores=1, paired=False):
    # Calculate total depth using flagstat.
    bam_paths = list(Path(base_path).glob('**/alignment.bam'))

    with ProcessPoolExecutor(max_workers=n_cores) as executor:
        results = executor.map(flagstat, bam_paths)
        flagstats = pd.DataFrame(
            dict(zip((fp.parent.name for fp in bam_paths), 
                     (res['count'] for res in results)))).T

    if paired:
        total = flagstats['both_mapped'] // 2
    else:
        total = flagstats['mapped'] - flagstats['secondary']

    total_reads = (
        total.to_frame('total_reads')
        .reset_index().rename(columns={'index': 'sample'}))
    
    # Calculate fusion statistics.
    fusion_paths = list(Path(base_path).glob('**/fusions.out'))

    fusion_stats = pd.DataFrame.from_records(
        ((fp.parent.name,) + star_fusion_stats(fp, 'T2onc') 
        for fp in fusion_paths),
        columns=['sample', 'fusion_reads', 'transposon_fusion_reads'])

    # Merge statistics per sample.
    merged = pd.merge(total_reads, fusion_stats, on='sample', how='left')
    
    # Add ratios.
    merged['ratio_reads_fusion'] = merged['fusion_reads'] / merged['total_reads']
    merged['ratio_fusions_transposon'] = (
        merged['transposon_fusion_reads'] / merged['fusion_reads'])
    
    return merged


def star_fusion_stats(file_path, transposon_name):
    junctions = read_chimeric_junctions(file_path)
    
    total_reads = junctions.shape[0]
    transposon_reads = junctions.loc[(junctions['seqname_a'] == transposon_name) ^ 
                                     (junctions['seqname_b'] == transposon_name)].shape[0]
    
    return (total_reads, transposon_reads)


sb_stats = read_stats('../data/interim/sb/star',
                      transposon_name='T2onc', 
                      n_cores=20,
                      paired=False)
sb_stats.head()

,sample,total_reads,fusion_reads,transposon_fusion_reads,ratio_reads_fusion,ratio_fusions_transposon
0,1566_10_11KOU023,15852706,12309,71,0.000776,0.005768
1,1566_11_11KOU024,14286991,13624,17,0.000954,0.001248
2,1566_12_11KOU028-L4,16686790,17971,43,0.001077,0.002393
3,1566_13_11KOU028-L5,17382889,17229,19,0.000991,0.001103
4,1566_14_11KOU029-R1,17553874,18934,57,0.001079,0.003010


In [27]:
sb_stats.mean() * 100

total_reads                 1.441677e+09
fusion_reads                1.444231e+06
transposon_fusion_reads     6.230894e+03
ratio_reads_fusion          9.932456e-02
ratio_fusions_transposon    4.218596e-01
dtype: float64

In [24]:
(sb_stats
 .rename(columns={
     'sample': 'Sample',
     'total_reads': 'Total reads',
     'fusion_reads': 'Fusion reads',
     'transposon_fusion_reads': 'Transposon fusion reads',
     'ratio_reads_fusion': 'Ratio fusion reads',
     'ratio_fusions_transposon': 'Ratio transposon fusions'
 })
.to_excel('../reports/supplemental/tables/table_s1_alignment_stats_sb.xlsx', index=False))

# Sanger dataset

In [16]:
sanger_stats = read_stats('../data/interim/sanger/star',
                          transposon_name='T2onc', 
                          n_cores=20,
                          paired=True)
sanger_stats.head()

,sample,total_reads,fusion_reads,transposon_fusion_reads,ratio_reads_fusion,ratio_fusions_transposon
0,TAPJ102_5c,82915344,597237,1533,0.007203,0.002567
1,TAPJ23_1a,89616265,520810,15,0.005812,0.000029
2,TAPJ23_1e,66460392,692985,1426,0.010427,0.002058
3,TAPJ23_2b,72623367,614396,3393,0.008460,0.005522
4,TAPJ47_4a,73234091,530312,983,0.007241,0.001854


In [25]:
(sanger_stats
 .rename(columns={
     'sample': 'Sample',
     'total_reads': 'Total pairs',
     'fusion_reads': 'Fusion pairs',
     'transposon_fusion_reads': 'Transposon fusion pairs',
     'ratio_reads_fusion': 'Ratio fusion pairs',
     'ratio_fusions_transposon': 'Ratio transposon fusions'
 })
.to_excel('../reports/supplemental/tables/table_sx_alignment_stats_sanger.xlsx', index=False))

In [29]:
sanger_stats.mean() * 100

total_reads                 7.688363e+09
fusion_reads                5.467412e+07
transposon_fusion_reads     2.317300e+05
ratio_reads_fusion          7.185204e-01
ratio_fusions_transposon    4.452995e-01
dtype: float64